In [1]:
%cd /teamspace/studios/this_studio/arabic_tts/inference/arabspeak

/teamspace/studios/this_studio/arabic_tts/inference/arabspeak


In [2]:
from IPython.display import Audio
import time
import onnxruntime as ort
from pathlib import Path
import numpy as np
import json
import re
from components.text import Characters, Tokenizer

In [3]:
path = Path("/teamspace/studios/this_studio/arabic_tts/training/export3")

In [4]:
model = path/"model.onnx"
config = path/"tokenizer_config.json"

In [5]:
with open(config,"r")as f:
    config = json.load(f)
tokenizer = Tokenizer.init_from_config(config)

In [6]:
session = ort.InferenceSession(model)

In [7]:
def prepare_inputs(session,inps):
    prepared = {}
    sess_inps = session.get_inputs()
    for i in range(len(sess_inps)):
        prepared[sess_inps[i].name] = inps[i]
    return prepared

In [8]:
texts = ["الشَّمْسُ تَغْرُبُ فِي الأُفُقْ، مُلَوِّنَةً السَّمَاءَ بِألْوَانٍ دَافِئَةٍ.",
         "يُرْجَى مُلَاحَظَةُ أَنَّ النَّصَّ الْمُرَادَ تَشْكِيلُهُ مَحْدُودٌ فِي عَدَدِ الأَحْرُفِ المَسْمُوحِ بِهَا. يُرْجَى مُلَاحَظَةُ أَنَّ النَّصَّ الْمُرَادَ تَشْكِيلُهُ مَحْدُودٌ فِي عَدَدِ الأَحْرُفِ المَسْمُوحِ بِهَا. ",
         "الْتَفَتَ الأَدَبُ يَنْقُلُ تَجَارِبَ الشُّعُوبِ وَحِكْمَتَهُمْ عَلَى مَرِّ الْعُصُورِ.",
         "وَرَجَحَتْ مَدْرَسَةْ سُعُودُ الثَانَوِيَّةُ ذلك .",
         "الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ الكَلََامْ "]

In [9]:
tokenizer.phonemizer._handle_special_words(texts[2])

'الْتَفَتَ الأَدَبُ يَنْقُلُ تَجَارِبَ الشُّعُوبِ وَحِكْمَتَهُمْ عَلَى مَرِّ الْعُصُورِ.'

In [30]:
tokenized = np.expand_dims(np.array(tokenizer.encode(texts[1])),[0])
scales = np.array([0.667,1,1]).astype(np.float32)
length = np.array([tokenized.shape[1]+10])
inp = [tokenized,length,scales]
prepared = prepare_inputs(session, inp)

In [31]:
s = time.time()
wav = session.run(None,prepared)[0][0][0]
t = time.time() - s

In [32]:
wav.shape[0]/t/16000

18.79242143509883

In [33]:
Audio(wav,rate=16000)